1. Bazując na bibliotekach pandas (https://github.com/pandas-dev/pandas) i faker (https://github.com/joke2k/faker), a konkretnie metodzie ssn z klasy Provider dla lokalizacji pl_PL (https://github.com/joke2k/faker/blob/master/faker/providers/ssn/pl_PL/__init__.py#L33) zaimplementuj funkcję o nazwie generate_ssns, która zwraca obiekt typu Series z liczbą rekordów określoną parametrem wejściowym funkcji oraz wartościami reprezentującymi wygenerowane losowe numery PESEL. 

In [1]:
import numpy as np
import pandas as pd
from faker import Faker

In [2]:
fake=Faker('pl_PL')
fake.name()

'pan Andrzej Kapuścik'

In [3]:
def generate_ssns(n):
    pesel_list=[]
    for i in range(n):
        pesel_list.append(fake.ssn())

    return pd.Series(data=pesel_list)

In [4]:
generate_ssns(10)

0    73022328832
1    99041796498
2    07310692116
3    70042699119
4    83020692679
5    88071774800
6    19220339986
7    77061636622
8    11250125852
9    86090223501
dtype: object

2. Zaimplementuj funkcję generate_unique_ssns (w dowolny sposób), która zwraca obiekt typu Series z liczbą rekordów określoną parametrem wejściowym funkcji oraz wartościami reprezentującymi wygenerowane losowe i unikalne (tylko w ramach zwracanej kolekcji) numery PESEL właściwe dla osób o płci (żeńska/męska) oraz urodzonych w przedziale dat (od-do) określonymi również parametrami wejściowymi tej funkcji. 

In [5]:
'''
dla osób urodzonych w latach 1900 do 1999 – miesiąc zapisywany jest w sposób naturalny, tzn. dwucyfrowo od 01 do 12

dla osób urodzonych w innych latach niż 1900–1999 dodawane są do numeru miesiąca następujące wielkości:

dla lat 2000–2099 – 20
dla lat 2100–2199 – 40
dla lat 2200–2299 – 60
dla lat 1800–1899 – 80

'''

def convert_ssn_to_date_int(some_ssn):
    d =some_ssn[4:6]
    m =some_ssn[2:4]
    y2 =some_ssn[0:2]
    
    if int(m) in range(1, 13):
        y1='19'
    elif int(m) in range(21, 33):
        y1='20'
        m=str(int(m)-20)
    elif int(m) in range(41, 53):
        y1='21'
        m=str(int(m)-40)
    elif int(m) in range(61, 73):
        y1='22'
        m=str(int(m)-60)
    elif int(m) in range(81, 93):
        y1='18'
        m=str(int(m)-80)
        
    return int(y1+y2+m+d)

In [6]:
convert_ssn_to_date_int('06243059966')

2006430

In [7]:
'06243059966'.isnumeric()

True

In [8]:
n=10
sex = 'f'
date_start_str = "19700101"
date_stop_str = "19750101"

def generate_unique_ssns(n,sex,date_start,date_stop):
    
    if not date_start.isnumeric() or not date_stop.isnumeric():
        return ('Error, date has to be numeric')
    
    elif len(date_start) != 8 or len(date_stop)!=8:
        return ('Error, check the length of the date')
    
    #converting dates into integers
    date_start_int = int(date_start)
    date_stop_int = int(date_stop)
    
    if date_stop_int < date_start_int:
        return('Error! date range not correct')

    #creating variable depending on sex
    ssn_list=[]
    if sex.lower()[0] == 'm':
        sex_variable=1
    elif sex.lower()[0] == 'f':
        sex_variable=0
    else:
        return('Error, please choose between male or female')

    #looping untill 'n' fake ssns are created
    while len(ssn_list)<n:
        
        #creating a fake ssn
        new_ssn = fake.ssn()
        date_int = convert_ssn_to_date_int(new_ssn)

        #validating sex
        if int(new_ssn[-2])%2 ==sex_variable:

            #validating date
            if  date_start_int <= date_int <= date_stop_int:
                
                #checking if unique
                if new_ssn not in ssn_list:
                    ssn_list.append(new_ssn)
                else:
                    pass
            else:
                pass
        else:
            pass

    return pd.Series(data=ssn_list)

In [9]:
generate_unique_ssns(100,'f','20210120','20221230')

0     21310629262
1     21310643828
2     21301579503
3     21312580125
4     21300425843
         ...     
95    21320576763
96    21320558783
97    21302510789
98    21312135004
99    21302938187
Length: 100, dtype: object

Następnie zaimplementuj wywołania funkcji generate_ssns oraz generate_unique_ssns dla 1 000, 10 000 i 100 000 rekordów, wskazując wybraną płeć oraz przedział dat urodzenia od 1990-01-01 do 1990-01-19. Dokonaj pomiaru i wyświetl czas trwania ich wykonania (osobno dla każdego wywołania każdej z tych dwóch funkcji). 


###Ze względu na ograniczoną moc obliczeniową komputera, zmieniłem wartości na 100, 1000, 3000###

In [14]:
import time
import pandas as pd

unique_list=[]
delta_list=[]
ssns_number_list=[]

for i in[1000,10000,100000]:

    start = time.time()
    generate_ssns(i)
    end = time.time()
    delta=(end - start)
    
    ssns_number_list.append(i)
    unique_list.append(0)
    delta_list.append(delta)

for i in[1000,10000,100000]:

    start = time.time()
    generate_unique_ssns(i,'f','19900101','19900119')
    end = time.time()
    delta=(end - start)
    
    ssns_number_list.append(i)
    unique_list.append(1)
    delta_list.append(delta)
    


df = pd.DataFrame(data ={'ssns_number':ssns_number_list,'unique':unique_list, "time elapsed [s]":delta_list})   

df


,ssns_number,unique,time elapsed [s]
0,100,0,0.021765
1,1000,0,0.065491
2,3000,0,0.114479
3,100,1,7.881444
4,1000,1,99.047192
5,3000,1,300.507328


4. Zaimplementuj funkcję o nazwie validate_ssn, która na wejściu przyjmuje numer PESEL wraz z określeniem oczekiwanej płci (żeńska/męska/dowolna) i daty urodzenia (konkretna/określona lub dowolna), a na wyjściu zwraca informację o poprawności numeru PESEL. Wewnątrz funkcji zawrzyj logikę weryfikującą poprawność syntaktyczną numeru PESEL z uwzględnieniem informacji o oczekiwanej płci oraz dacie urodzenia. Następnie przetestuj działanie funkcji validate_ssn na przykładowych danych.

In [11]:
def validate_ssn(ssn, sex, birth_date):
    
    """
    
    validates given ssn with given sex ('female/male') and birth date in format YYYYMMDD
    
    Returns 1 if validation is positive and 0 if negative
    
    """

    if sex.lower()[0] == 'm':
        sex_variable=1
    elif sex.lower()[0] == 'f':
        sex_variable=0
    else:
        return('please choose sex between male or female')
    
    
    if not birth_date.isnumeric():
        return('Error, please provide birth date in format YYYYMMDD')
    elif len(birth_date) != 8:
        return('Error, please provide birth date in format YYYYMMDD')
    else:
        pass

    #converting ssn 
    date_int = convert_ssn_to_date_int(ssn)

    #validating sex
    if int(ssn[-2])%2 == sex_variable:

        #validating date
        if date_int == int(birth_date):
            
            return 1
        else:
            return 0
    else:
        return 0

In [12]:
#validating a proper ssn:
validate_ssn('21010176481', 'female', '19210101')

1

In [13]:
#validating a false ssn:
validate_ssn('21010176481', 'male', '19210101')

0